# Scrapping data

En este fichero se realizara la obtención de los datos de concentraciones de contaminantes atmosféricos proporcionados por la estaciones de medición del ayuntamiento de Madrid.

### Librerías

In [1]:
import os
import re
import requests
import urllib.request
from bs4 import BeautifulSoup
from zipfile import ZipFile
import fileinput

### Tratamiento de los datos

Se establece la fuente principal del proyecto

In [2]:
fuente_principal=!pwd

Enlace de la web de datos de calidad del aire horario del ayuntamiento de madrid 

In [3]:
enlace = "https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"

Se realiza una llamada a la página

In [4]:
url = urllib.request.urlopen(enlace)
CalidadAire=BeautifulSoup(url.read(),'lxml')

Se genera un diccionario vacío, donde se albergará los años como llave y en los enlaces de descarga como valares. Se tendrá que especificar los años que se quieran estudiar

In [5]:
dict_link={}
ano_inicio=2015
ano_fin=2019

Se busca la etiqueta en la que está alojada la información de los datos de calidad del aire horario, y se extrae el año y el enlace de descarga de los ficheros, para guardarlo en el diccionario antes creado.

In [6]:
links=CalidadAire.find_all("a", class_="asociada-link ico-zip")
for line in links:
    link='https://datos.madrid.es'+line.get('href')
    ano=int(line.find_previous("p", class_="info-title").string)
    if (ano >= ano_inicio) & (ano <= ano_fin):
        dict_link[ano]=link

Al tener ya un diccionario con los años y los enlaces de descarga de datos horarios para cada año, se procederá a realizar la descarga de los datos para los años especificados anteriormente

Pero antes de todo se crea la carpeta "data" y una subcarpeta "source" donde se almacenará toda la información descargada

In [7]:
fuente_datos='./data/source'

if os.path.isdir(fuente_datos):
    print('el directorio ya existe')
else:
    os.makedirs(fuente_datos)
    print('Se han creado el directorio')

os.chdir(fuente_datos)

el directorio ya existe


Se comienza con la descarga de los datos 

In [16]:

for key,value in dict_link.items():
        print("Descargando la información del año %s..."%format(key))
        zipname=str(key)+".zip"
        urllib.request.urlretrieve(value, zipname)
        with ZipFile(zipname, 'r') as zipObj:
            # Coge una lista de todos archivo del fichero ZIP
            listOfFileNames = zipObj.namelist()
            # Itera sobre los nombre de los ficheros
            for fileName in listOfFileNames:
                # Comprueba que tiene una terminación .txt
                if fileName.endswith('.txt'): 
                    # Extrae el archivo
                    zipObj.extract(fileName, './')
        
        print("¡Año %s completado!"%format(key))

os.remove('.DS_Store')

Descargando la información del año 2019...
¡Año 2019 completado!
Descargando la información del año 2018...
¡Año 2018 completado!
Descargando la información del año 2017...
¡Año 2017 completado!
Descargando la información del año 2016...
¡Año 2016 completado!
Descargando la información del año 2015...
¡Año 2015 completado!


Cambio de nombre de los ficheros que albergan los datos de contaminación del aire, para que todos tengan la estructura yymm.txt

In [17]:
mes={"ene_mo":"01","ene":"01",
    "feb_mo":"02","feb":"02",
    "mar_mo":"03","mar":"03",
    "abr_mo":"04","abr":"04",
    "may_mo":"05","may":"05",
    "jun_mo":"06","jun":"06",
    "jul_mo":"07","jul":"07",
    "ago_mo":"08","ago":"08",
    "sep_mo":"09","sep":"09",
    "oct_mo":"10","oct":"10",
    "nov_mo":"11","nov":"11",
    "dic_mo":"12","dic":"12"}

In [18]:
for file in os.listdir():
    res = re.findall(r'[0-9]+', file)
    if len(str(res))==8:
        os.remove(file)
        pass
    else:
        if res[0]=='17':
            new_name=res[0]+mes[file[0:3]]+".txt"
            os.rename(file,new_name)
        else:
            new_name=res[0]+mes[file[0:6]]+".txt"
            os.rename(file,new_name)
    

1501.txt
['1501']
1502.txt
['1502']
1503.txt
['1503']
1504.txt
['1504']
1505.txt
['1505']
1506.txt
['1506']
1507.txt
['1507']
1508.txt
['1508']
1509.txt
['1509']
1510.txt
['1510']
1511.txt
['1511']
1512.txt
['1512']
1601.txt
['1601']
1602.txt
['1602']
1603.txt
['1603']
1604.txt
['1604']
1605.txt
['1605']
1606.txt
['1606']
1607.txt
['1607']
1608.txt
['1608']
1609.txt
['1609']
1610.txt
['1610']
1611.txt
['1611']
1612.txt
['1612']
1701.txt
['1701']
1702.txt
['1702']
1703.txt
['1703']
1704.txt
['1704']
1705.txt
['1705']
1706.txt
['1706']
1707.txt
['1707']
1708.txt
['1708']
1709.txt
['1709']
1710.txt
['1710']
1711.txt
['1711']
1712.txt
['1712']
1801.txt
['1801']
1802.txt
['1802']
1803.txt
['1803']
1804.txt
['1804']
1805.txt
['1805']
1806.txt
['1806']
1807.txt
['1807']
1808.txt
['1808']
1809.txt
['1809']
1810.txt
['1810']
1811.txt
['1811']
1812.txt
['1812']
1901.txt
['1901']
1902.txt
['1902']
1903.txt
['1903']
2015.zip
['2015']
2016.zip
['2016']
2017.zip
['2017']
2018.zip
['2018']
2019.zip
[

Debido a que la información en todos los ficheros no es la misma, se van a a tener que modificar alguno de ellos, en concreto los anteriores a Octubre de 2017. Los archivos que se tienen que modificar tienen el incoveniente de que no están separados por comas, cosa fundamental para la limpieza de los datos posterior.

En las siguientes lineas se van a añadir comas en aquellas posiciones que el resto de archivos, cuya modificación no es necesaria, la tienen

In [19]:
for file in os.listdir():
    if file<'1710.txt':
        for line in fileinput.FileInput(file,inplace=1):
            comas=[2,5,8,10,12,14,16,18,20]
            for i,j in enumerate(comas):
                line=list(line)
                line.insert(i+j,',')
            line=line[0:-1]
            line="".join(line)
            line=re.sub(r"[^V.,\-(0-9)]","N",line)
            line=line.replace("V",",V,")
            line=line.replace("N",",N,")          
            if line[len(line)-1]==',': line=line[0:(len(line)-1)]
            print(line)